In this programming assignment, we're going to be implementing an MDP, Value Iteration and Policy Iteration, and at Belief State Updates for a POMDP.

# MDP formulation

The first step is to formulate the MDP using a Transition function, T, and a Reward function, R. Use the templates below and the parameters from problem 2 in the written component to implement T and R.

## Transition function

In [ ]:
def T(s, a, s_):
    """Transition function from state s to state s_ via action a."""
    
    if s == 'healthy':
        
        if a == 'vegetables':
            if s_ == 'healthy':
                return 1
            elif s_ == 'sick':
                return 0
            elif s_ == 'toothless':
                return 0
            else:
                print('Specify valid state for s_: healthy, sick, or toothless')
                return
            
        elif a == 'candy':
            if s_ == 'healthy':
                return 0.25
            elif s_ == 'sick':
                return 0.75
            elif s_ == 'toothless':
                return 0
            else:
                print('Specify valid state for s_: healthy, sick, or toothless')
                return

        else:
            print('Specify valid action for a: candy or vegetables')
            return
    
    elif s == 'sick':
        
        if a == 'vegetables':
            if s_ == 'healthy':
                return 0.25
            elif s_ == 'sick':
                return 0.75
            elif s_ == 'toothless':
                return 0
            else:
                print('Specify valid state for s_: healthy, sick, or toothless')
                return
            
        elif a == 'candy':
            if s_ == 'healthy':
                return 0
            elif s_ == 'sick':
                return 0.875
            elif s_ == 'toothless':
                return 0.125
            else:
                print('Specify valid state for s_: healthy, sick, or toothless')
                return
        else:
            print('Specify valid action for a: candy or vegetables')
            return

    elif s == 'toothless':
        if s_ != 'toothless':
            return 0
        elif s_ == 'toothless':
            return 1
    
    else:
        print('Specify valid state for s: healthy, sick, or toothless.')
        return 

## Reward function

In [ ]:
def R(s, a, s_):
    
    if s == 'toothless':
        return 0
    
    if a == 'vegetables':
        return 4
            
    elif a == 'candy':
        return 10

# MDP Solvers

Now that you've implemented T and R, implement Value Iteration and Policy Iteration using the templates below and the pseudocode presented in class or in the book.

## Value Iteration

In [ ]:
S = ["healthy", "sick", "toothless"]
A = ["candy", "vegetables"]
epsilon = 0.0001
gamma = 1

def valueIteration(S, A, T, R, gamma, epsilon):

    # initialize dict for Q 
    Q = {}
    for s in S:
        for a in A:
            Q[s+"-"+a] = 0

    while True:
        # create temp dict Qnew
        Qnew = Q.copy()

        # initialise delta to 0
        delta = 0
        
        # loop over state space 
        for s in S:
            for a in A:
                # update optimal policies using bellman equation
                Qnew[s+"-"+a] = sum(T(s, a, "-") * (R(s, a, "-") + gamma * Q[s+"-"+a]))
                # update delta 
                delta = max(delta, abs(Qnew[s] - Q[s]))
                # store in Q
                Q = Qnew.copy()
        print(Q)
        if delta < epsilon * (1 - gamma) / gamma:
            return Q   

    pi = Q.copy() 
    return pi 


## Policy Iteration

In [ ]:
S = ["healthy", "sick", "toothless"]
A = ["candy", "vegetables"]
epsilon = 0.0001
gamma = 0.9

def policyIteration(S, A, T, R, gamma, epsilon):

    # initialise dict for V and pi
    V = {}
    pi = {}
    for s in S:
        V[s] = 0
        pi[s] = A[0]
    
    while True:

        # policy evaluation
        delta = 0

        for s in S:
            Vprime[s] = V[s]
            # evaluate using fixed policy using bellman equation
            V[s] = R(s, pi[s], "") + gamma * sum((T(s, pi(s), "") * Vprime[s]))
            delta = max(delta, abs(V[s] - Vprime[s]))
       
        if delta < epsilon * (1 - gamma) / gamma:
            return V
       
        unchanged = True
        
        U = V.copy

        # extract better policy based on these new values
        for s in S:
            for a in A:
                Vprime[s] = U[s]
                pi[s] = R(s, a[s], "") + gamma * sum((T(s, a[s], "") * Vprime[s]))
                delta = max(delta, abs(pi[s] - Vprime[s]))
        if delta < epsilon * (1 - gamma) / gamma:
            return pi
        
        for s in S:
            if V[s] != pi[s]:
                pi[s] = V[s]
                unchanged = False
        if unchanged:
            return pi

## Finding the $\gamma$ threshold

You may have noticed that the optimal policy is typically to eat candy in every state. Find the smallest, up to 0.0001 error, discount factor $\gamma$ such that for $\pi$, the optimal policy, $\pi$(sick) = vegetables.

In [ ]:
def find_gamma(valueIteration, epsilon=0.0001):
    
    # equated calculations from question 2 b of the written part to solve for Vpi2(s) - Vpi1(s) = epsilon, then rearranged in terms of gamma and computed below

    Sick_Candy = valueIteration('sick', 'candy', "", "", "", epsilon)
    Sick_Vegetables = valueIteration('sick', 'vegetables', "", "", "", epsilon)
    Healthy_Vegetables = valueIteration('healthy', 'vegetables', "", "", "", epsilon)
    gamma_min = (24 + 8 * epsilon) / (2 * Healthy_Vegetables + 6 * Sick_Vegetables + 7 * Sick_Candy)
    return gamma_min

# POMDP Belief States

In problem 3 of the written you computed belief state updates by hand. To do this you had to compute the evidence posterior $\mathbb P(e | s)$ and run the FORWARD updates $$b(s') = \alpha \mathbb P(e|s') \sum_{s \in S} \mathbb P(s' | s, a) b(s).$$
Implement the evidence posterior, O, below and the update_belief_state method using the FORWARD update. You can use this to check your answer for problem 3(c) in the written.

## Evidence posterior

In [ ]:
def O(evidence, state):

    if evidence == 'F':
        if state == 'toothless':
            return 1
        else:
            return 0
            pass
    else:
        temperature = evidence
        # State Posteriors
        P_healthy_given_T_minus = 0.9
        P_sick_given_T_minus = 0.1
        
        # Evidence Posteriors
        P_healthy_given_T_plus = 0.01 
        P_sick_given_T_plus = 0.99 
        
        # Evidence Priors
        P_T_minus = (84/89) 
        P_T_plus = (5/89) 
        
        # State Priors
        P_healthy = 0.85 
        P_sick = 0.15 

        if temperature == 0:
            
            P_temperature = P_T_minus
            
            if state == 'healthy':
                P_state_given_temperature = P_healthy_given_T_minus
                P_state = P_healthy
            elif state == 'sick':
                P_state_given_temperature = P_sick_given_T_minus
                P_state = P_sick
            elif state == 'toothless':
                P_state_given_temperature = 0
                P_state = 0
                pass
            else:
                print('Specify healthy or sick for state!')
                return
            
        elif temperature == 1:

            P_temperature = P_T_plus
            
            if state == 'healthy':
                P_state_given_temperature = P_healthy_given_T_plus
                P_state = P_healthy
            elif state == 'sick':
                P_state_given_temperature = P_sick_given_T_plus
                P_state = P_sick
            elif state == 'toothless':
                P_state_given_temperature = 0
                P_state = 0
            else:
                print('Specify healthy or sick for state!')
                return
        else:
            print('Specify binary variable for temperature!')
            return
        
        if evidence != 'F':
            if state != 'toothless':
                P_temperature_given_state = (P_state_given_temperature * P_temperature)/(P_state)
                return P_temperature_given_state 
            else:
                return 0



## Belief state update 

In [22]:
epsilon=0.0001


def update_belief_state(temperature, action, belief_state):

    # initialize belief state tuple --> b(s)
    BS = belief_state

    # initialise evidence given new belief state --> P(e|s')
    if temperature == 0:
        P_evidence_given_s  = O(0, 'sick')
        P_evidence_given_h  = O(0, 'healthy')
    elif temperature == 1:
        P_evidence_given_s  = O(1, 'sick')
        P_evidence_given_h  = O(1, 'healthy')
    elif temperature == 'F':
        P_evidence_given_t = 1

    # initialise new belief state given belief state and action --> P(s'|s,a)
    if action == 'candy':
        P_s_given_h = T('sick', 'candy', 'healthy')
        P_s_given_s = T('sick', 'candy', 'sick')
        P_h_given_h = T('healthy', 'candy', 'healthy')
        P_h_given_s = T('healthy', 'candy', 'sick')
    if action == 'vegetables':
        P_s_given_h = T('sick', 'vegetables', 'healthy')
        P_s_given_s = T('sick', 'vegetables', 'sick')
        P_h_given_h = T('healthy', 'vegetables', 'healthy')
        P_h_given_s = T('healthy', 'vegetables', 'sick') 

    A = P_evidence_given_h * (P_h_given_h * BS[0] + P_s_given_h * BS[1])
    B = P_evidence_given_s * (P_h_given_s * BS[0] + P_s_given_s * BS[1])
    b_0 = A/(A+B)
    b_1 = B/(A+B)
    U = (b_0, b_1, 0)
    
    updated_belief_state = U
    return updated_belief_state 